In [1]:
import glob 
import os
from PIL import Image
import matplotlib.pyplot as plt
from PIL import ImageFile
import numpy as np
import json
%matplotlib inline
from tqdm import tqdm

In [2]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
# di_class = ['Ab01','Ab02','Ab03','Ab04','Ab05','Ab06','Ab07','Ab08','Ab09','Ab10']
di_class = ['Ab01','Ab02','Ab05','Ab07','Ab08']

In [4]:
img_dir_path = 'C:/Users/ejkan/Desktop/abdnominal_data/validation/dog/img'


In [5]:
json_dir_path = "C:/Users/ejkan/Desktop/abdnominal_data/validation/dog/label"
json_name_list = os.listdir(json_dir_path)

In [6]:
print(len(json_name_list))

30295


In [7]:
print(json_name_list[:10])

['D_60_20120612_CM_0028_NOR_Ab07_20211221_0730.json', 'D_60_20120612_CM_0028_NOR_Ab08_20211120_0093.json', 'D_60_20120612_CM_0028_NOR_Ab09_20211221_0085.json', 'D_60_20120612_CM_0032_ABN_Ab09_20211217_1119.json', 'D_60_20120612_CM_0032_NOR_Ab08_20211228_0725.json', 'D_60_20120612_CM_0032_NOR_Ab09_20211224_3450.json', 'D_60_20120612_CM_0032_NOR_Ab09_20211224_3451.json', 'D_60_20120612_IF_0020_ABN_Ab10_20210124_0148.json', 'D_60_20120612_IF_0020_NOR_Ab01_20210124_0001.json', 'D_60_20120612_IF_0020_NOR_Ab01_20210127_0001.json']


In [8]:
def padding(img_path):
    img = Image.open(img_path)
    img = img.convert('RGB')
    w, h = img.size
    img = np.array(img)
    
    margin = [np.abs(w - h) // 2 , np.abs(w - h) // 2]
    if w > h:
        margin_list = [margin, [0,0]]
    else:
        margin_list = [[0,0], margin]
        
    if len(img.shape) == 3:
        margin_list.append([0,0])
        
    output = np.pad(img, margin_list, mode='constant')
    img = Image.fromarray(output)
    img = img.resize((224, 224))
    img = np.array(img)
    return img

In [9]:
def status_to_int(status):
    if status == 'NOR': #정상
        int_status = 0
    else:
        int_status = 1
    return int_status

In [10]:
def json_name_to_img_path(json_name):
    img_name = json_name.split('.')[0] +'.jpg'
    img_path = img_dir_path + '/'+ img_name
    return img_path

In [11]:
dir_path = "C:/Users/ejkan/Desktop/data_224_dog_val"

for di in di_class:
    x_data, y_data = [], []
    for json_name in tqdm(json_name_list):
        if json_name.split('_')[6] == di:
            json_path = json_dir_path + '/'+ json_name
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                if data["metadata"]['Position'] == "VD":
                    continue
                status = data["metadata"]['Disease']
                int_status = status_to_int(status)
                y_data.append(int_status)
            img_path = json_name_to_img_path(json_name)
            img = padding(img_path)
            x_data.append(img)
    x_data = np.array(x_data)
    y_data = np.array(y_data)

    np.save(dir_path +'/'+di+'_x_data_lateral.npy', x_data)
    np.save(dir_path +'/'+di+'_y_data_lateral.npy', y_data)
    

100%|██████████| 30295/30295 [03:31<00:00, 143.47it/s]
